In [283]:
from diffbot_api import *
from ttl import *

from converter import URIConverter
from math import log
import json
from utils import truncated_log, overlap
from candidate import Candidate

    
phrases = "New York, windows, catwalk, teardrops, commuters, curved, floor, shape, video, walls, Grand Central Terminal"
phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
context = "Inside, it’s even wackier: curved walls, windows in the shape of teardrops, and a catwalk with a tiny video screen embedded in the floor that shows an endless loop of antlike commuters rushing through Grand Central Terminal in New York."


class BaselineLinker(object):
    def __init__(self, use_overlap = True, verbose = True):
        self._cq = CachedQuery()
        self._conv = URIConverter()
        self._use_overlap = use_overlap
        self._verbose = verbose
        
    def __del__(self):
        self.close()
        
    def close(self):
        try:
            self._cq.close()
            self._conv.close()
        except:
            print("Warning: trying to close a closed object.") 
    
    def find_wiki_uri(self, uris):
        for uri in uris:
            if "wikipedia.org" in uri:
                return uri

        return None

    def get_dbpedia_uri(self, candidate):
        dbpedia_uri = None
        if candidate.wiki is not None:
            wiki_uri = candidate.wiki
            dbpedia_uri = conv.wikipedia2dbpedia(wiki_uri)
        else:
            for uri in candidate.uris:
                dbpedia_uri = conv.wikidata2dbpedia(uri)
                #print(uri, dbpedia_uri)
                if dbpedia_uri is not None: break

        return dbpedia_uri

    def link_db_query(self, target, diffbot_query_response, use_overlap=True):
        candidates = []
        if "data" not in diffbot_query_response:
            return candidates
        else:
            data = diffbot_query_response["data"]

        for hit in data:
            uris = set(hit["allUris"])
            if "origin" in hit: uris.add( hit["origin"] )
            if "origins" in hit: uris.union( set(hit["origins"]) )
            if "wikipediaUri" in hit:
                uris.add( hit["wikipediaUri"] )

            if "importance" in hit:
                name = hit["name"]
                importance = float(hit["importance"])
                score = truncated_log(importance) * self.overlap(name, target) if use_overlap else importance
                c = Candidate(score,
                              name,
                              find_wiki_uri(uris),
                              hit["types"],
                              hit["allNames"],
                              uris)
                candidates.append(c)
            else:
                print("Warning: Skipping a hit without importance value.")

        return sorted(candidates, reverse=True)

    def link(self, context, phrases):
        best = []
        for phrase in phrases:
            candidates = []
            for entity_type in EL_POL_ENTITY_TYPES:
                r = self._cq.make_query('type:{} name:"{}"'.format(entity_type, phrase.text))
                db_response = json.loads(r.content)
                candidates += self.link_db_query(phrase.text, db_response, use_overlap=self._use_overlap) 
            candidates = set(candidates)

            best.append( (phrase, sorted(candidates, reverse=True)[0]) )
            dbpedia_uri = get_dbpedia_uri(best)

            if self._verbose:
                print("{} {}: {}: {}".format(phrase.text, best.score, best.name, dbpedia_uri), end="\n")

        if len(best) != len(phrases):
            print("Warning: length of output is not equal to length of input {} != {}".format(len(best), len(phrases)))
        
        return best
    
    def link_ttl(self, input_ttl):
        graph, context, phrases = parse_d2kb_ttl(input_ttl)

        print("# triples input:", len(graph))
        for linked_phrase in self.link(context, phrases):
            graph.add( (phrase.subj, CLASS_URI, NONE_URI) )
            graph.add( (phrase.subj, LINK_URI, NONE_URI) )
        
        print("# triples output:", len(graph))
        output_ttl = str(graph.serialize(format='n3', encoding="utf-8"), "utf-8")
        return output_ttl



New York 8.824677891164198: New York: http://dbpedia.org/resource/New_York_(state)
Grand Central Terminal 6.396929655216146: Grand Central Terminal: None


In [273]:
max(1,2)

2

In [ ]:
gct
russians

In [19]:
%load_ext autoreload
%autoreload 2

# DQL

In [ ]:
query_and_save('allUris:"barackobama.com"', "data/all-uris.json")
query_and_save('wikipediaUri:"en.wikipedia.org/wiki/Barack_Obama"', "data/wiki-uri.json")
query_and_save('allUris:"en.wikipedia.org/wiki/Barack\_Obama"', "data/all-uris-wiki.json")
query_and_save('origins:"en.wikipedia.org/wiki/Barack_Obama"', "data/origins.json")

In [ ]:
for entity_type in entity_types:
    query_and_save(
        query='type:{}'.format(entity_type),
        output_fpath="data/{}.json".format(entity_type))
    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")


# Testing type of links

In [ ]:
query_and_save(
    query='type:Person name:"Angela Merkel"',
    output_fpath="data/am.json")

query_and_save(
    query='type:Person name:"Barack Obama"',
    output_fpath="data/bo.json")

query_and_save(
    query='type:Person name:"Nicolas Sarkozy"',
    output_fpath="data/ns.json")

query_and_save(
    query='type:Person name:"Diego Maradona"',
    output_fpath="data/dm.json")